# _d_-invariants of 2-bridge links

The d invariant of a 2-bridge link is denoted $d(S(p,q))$ and is a collection of $p$ rational numbers

$$d(S(p,q),i) \hspace{1cm} for \hspace{0.2cm} i\in[0,1,...,p-1].$$

The _d_ invariants can be computed by the following recursive procedure as in ??? from ???. We are given that $p > 0$.

 1. $d(S(1,q),i) = 0$ for all $q$ and all $i$.
 
 2. Let $x\%p$ denote the integer between $0$ and $p$ which is congruent to $x$ modulo $p$, then $d(S(p,q),i) = d(S(p,q\%p),i\%p)$.
 
 3. And for $p > q > 0$ and $0 \le i < p + q$,
 
 $$d(S(p,q),i) = \frac{(2i + 1 - p - q)^{2} - pq}{4pq} - d(S(q,p),i)\hspace{1cm}$$
 
We use this procedure to define a recursive function `dS` to compute the _d_ invariant given $p > 0$ and $q,i \in \mathbb{Z}$. We start by checking for bad input, then reduce $q,i$ modulo $p$ before implementing our recursive step. The recursion terminates when $p = 1$ as given in step 1. We assume that this occurs in finite iterations.

We will use the data type `Fraction` from the `fractions` module in python to avoid rounding errors. To make it work with our framework, we start by writing a function `extract_tuple` to extract a positive numerator and  integer denominator from our fraction. The fraction module ensures that $p$ and $q$ are coprime integers.

In [31]:
import numpy as np
from fractions import Fraction

def extract_tuple(frac):
    if (frac < 0):
        p = -frac.numerator
        q = -frac.denominator
    else:
        p = frac.numerator
        q = frac.denominator
    return p,q

# Recursive d-invariant
def d(S,i):
    # checking input values are positive integers
    if ((not isinstance(S, Fraction))):
        raise Exception('S must be a fraction. S was {}.'.format(S))
        
    if ((not isinstance(i, int))):
        raise Exception('i must be an integer. i was {}.'.format(i))
    
    p,q = extract_tuple(S)
    # we terminate if p = 1 (using (1))
    if (p == 1):
        return Fraction(0,1)

    q = q%p
    i = i%p                                                          # these steps combine (2) and (3)
    r = Fraction(((((2*i) + 1 - p - q)**2) - p*q),(4*p*q))
    # our recursive step is safe since we have checked for bad input and reduced q and i mod p.
    return r - dS(q,p,i)

## Listing the invariant as a numpy array

In this section we will list all the invariants for a given rational number. We have implemented our _d_ invariant function above using using the following fact from the project description.

$$
d(S(p,q),i) = d(S(p,q),i+np)
$$

for all $n \in \mathbb{Z}$. This simplifies our task of listing the invariants for a number $\frac{p}{q}$. We can simply list $d(S(p,q),i)$ for $i = 0,1,2, \dots p-1$. Any other value of $i$ can thus be reduced modulo $p$ to fall into our listed range. The code is a simple for-loop iterating over these values of $i$ and placing the corresponding fraction $d(S(p,q),i)$ into a numpy array. The type of the numpy array is `obj` to allow for fractions, thus avoiding rounding errors. The error checking falls onto the function `d` implemented above, which raises exceptions for bad input.

In [32]:
def list_invariants(S):
    p = abs(S.numerator)
    A = np.zeros(p).astype('object')
    for i in range(p):
        A[i] = (d(S,i))
    return A

To test out our function we try $p = 9$ and $q = 7$ and pretty print the output.

In [33]:
for r in list_invariants(Fraction(9,7)):
    print(r)

0
-2/9
4/9
0
4/9
-2/9
0
-8/9
-8/9


This seems to agree with the sample output we are given in the project description.

## _d_-invariants of equivalent links

We have already seen that there are cases where two 2-bridge links are equivalent despite having different values of $q$; i.e. $S(p,q_{1}) = S(p,q_{2})$.
And we verified that this is true with the following cases:

$$S(p,q) = S(p,q + np) \hspace{1cm} for \hspace{0.2cm} n\in \mathbb{Z}$$

Here we shall consider what the above equality means for the _d_ invariance.
The function below takes $p$ and $q$ as input and outputs the _d_ invariance lists for the ten smallest positive values of $q$ of the form $q + np$.

In [36]:
# Function that takes p as input and ouputs all d invariance lists of S(p,q + np) for (q + np) > 0
# Have chosen to take the first 10 such values for q to give a good sample of the pattern produced
def dS_congruent_q(p,q):
    
    while q - p > 0:
        q = q - p             # Sets q to the smallest positive value of q of the form q + np 
    
    B = []                    # We create a list of the d invariance lists for each of the first ten q values
    Q = []                    # And a list of said values of q
    
    while q < 10*p:           # We are only finding the first ten lists
        B.append(list_invariants(Fraction(p,q)))
        Q.append(q)
        q = q + p
    
    print('The first ten values of q for which S(',p,',q) is equivalent are', Q)
    
    for i in range(len(B)):
        print('The d invariance list for q =',Q[i],'is',B[i])
        print('=============================================')
    
    return 'It is safe to assume that this pattern continues for infinte values of q'

We will again check our function against our known example of $d(S(9,7))$:

In [37]:
print(dS_congruent_q(9,7))

The first ten values of q for which S( 9 ,q) is equivalent are [7, 16, 25, 34, 43, 52, 61, 70, 79, 88]
The d invariance list for q = 7 is [Fraction(0, 1) Fraction(-2, 9) Fraction(4, 9) Fraction(0, 1)
 Fraction(4, 9) Fraction(-2, 9) Fraction(0, 1) Fraction(-8, 9)
 Fraction(-8, 9)]
The d invariance list for q = 16 is [Fraction(0, 1) Fraction(-2, 9) Fraction(4, 9) Fraction(0, 1)
 Fraction(4, 9) Fraction(-2, 9) Fraction(0, 1) Fraction(-8, 9)
 Fraction(-8, 9)]
The d invariance list for q = 25 is [Fraction(0, 1) Fraction(-2, 9) Fraction(4, 9) Fraction(0, 1)
 Fraction(4, 9) Fraction(-2, 9) Fraction(0, 1) Fraction(-8, 9)
 Fraction(-8, 9)]
The d invariance list for q = 34 is [Fraction(0, 1) Fraction(-2, 9) Fraction(4, 9) Fraction(0, 1)
 Fraction(4, 9) Fraction(-2, 9) Fraction(0, 1) Fraction(-8, 9)
 Fraction(-8, 9)]
The d invariance list for q = 43 is [Fraction(0, 1) Fraction(-2, 9) Fraction(4, 9) Fraction(0, 1)
 Fraction(4, 9) Fraction(-2, 9) Fraction(0, 1) Fraction(-8, 9)
 Fraction(-8, 9)]
The

## _d_-invariant of inequivalent links

Now we will turn our attention to testing the converse; whether inequivalent links can produce the equal invariants. More formally, we wish to test

$$S(p,q_1) \neq S(p,q_2) \implies d(S(p,q_1),i) \neq d(S(p,q_2),i)$$

for all $i = 0,1, \dots, p-1$.

We thus need a way of testing the equivalence of two 2-bridge links in our program. The previous sections on 2-bridge links gives us some insight into when 2-bridge links are equivalent. However to cover all cases of equivalence rigorously we will use thm ??? from ???. This is stated below in our nomenclature and notation without proof.

___Theorem___    _2-bridge links $S(p_1,q_1)$ and $S(p_2,q_2)$ are equivalent if and only if $p_1 = p_2$_ and either $q_1 \equiv q_2$ mod $p_1$ or $q_1 q_2 \equiv 1$ mod $p_1$.

We use this to design our equivalence test as follows. The function takes two fractions as input and returns a boolean value. Error checking is implicit in the `Fraction` data type. $p_1$ and $p_2$ must remain positive so the first step pushes the minus sign from the numerator to the denominator if required.

In [38]:
def equivalent_link(S1,S2):
    # type checks
    if ((not isinstance(S1, Fraction))):
        raise Exception('S1 must be a fraction. S1 was {}.'.format(S1))
    if ((not isinstance(S2, Fraction))):
        raise Exception('S2 must be a fraction. S2 was {}.'.format(S2))
        
    p1,q1 = extract_tuple(S1)
    p2,q2 = extract_tuple(S2)
    
    # testing whether the corresponding links are equivalent
    if  not (p1 == p2):
        return False		# numerators must be equal
    
    A = (q1%p1 == q2%p1)
    B = ((q1*q2)%p1 == 1)
    return (A ^ B)			# and either A or B

Our strategy for the test will be to use nested for-loops. We use the fact that $S(p,q_1) = S(p,q_2) \iff S(p,q_2) = S(p,q_1)$ to limit the range of `q2`. Also the range for `q1` can be limited by the fact from 2(c) of 2-bridge links. We use a function so that we can `return` to break out of the nested for loops.

The function `list_invariants` involves multiple calls to a recursive function and is clearly computationally heavy. Thus to minimise calls to it, we will only call it if $S(p,q_1) \neq S(p,q_2)$.

Despite this, the program is still slow and we will thus only test for $p \leq 20$ (which tests $\sum_{i=1}^{20} \frac{i(i+1)}{2}$ pairs of links). If we test all links where $p \leq 1000$ we would be testing more than sixteen million pairs. The maximum value of $p$ can be changed in the function below if required.

In [49]:
def invariance_implies_equivalence(n):
    for p in range(1,n+1):
        for q1 in range(1,p):
            for q2 in range(1,q1):
                # We are testing whether links_unequal and not d_unequal (negation of our implication).
                links_unequal = not equivalent_link(Fraction(p,q1), Fraction(p,q2))
                if links_unequal:
                    d_unequal = not np.array_equal(list_invariants(Fraction(p,q1)),list_invariants(Fraction(p,q2)))
                    if not d_unequal:
                        return False
                
    return True
            
predicate = invariance_implies_equivalence(20)
    
if predicate:
    print('statement is verified for p <= 20')
else:
    print('exceptions to the statement were found')

statement is verified for p <= 20


## Testing the property of slice knots

When $m < 1$ want to check for which $q = 1,2, \dots m^2-1$ the invariant list has $m$ zeroes. This is implemented using another simple nested for-loop.

In [55]:
slice_property = []

for m in range(1,12):
    for q in range(1,m**2):
        if (np.count_nonzero(list_invariants(Fraction(m**2, q))) == (m**2 - m)):
            slice_property.append(Fraction(m**2,q))
            
# Removing duplicates
slice_property = list(dict.fromkeys(slice_property))

print(slice_property)

[Fraction(4, 1), Fraction(2, 1), Fraction(4, 3), Fraction(9, 2), Fraction(9, 4), Fraction(9, 5), Fraction(9, 7), Fraction(16, 3), Fraction(16, 5), Fraction(16, 7), Fraction(16, 9), Fraction(16, 11), Fraction(16, 13), Fraction(25, 4), Fraction(25, 6), Fraction(25, 7), Fraction(25, 9), Fraction(25, 11), Fraction(25, 14), Fraction(25, 16), Fraction(25, 18), Fraction(25, 19), Fraction(25, 21), Fraction(36, 5), Fraction(36, 7), Fraction(36, 11), Fraction(36, 13), Fraction(36, 23), Fraction(36, 25), Fraction(36, 29), Fraction(36, 31), Fraction(49, 6), Fraction(49, 8), Fraction(49, 13), Fraction(49, 15), Fraction(49, 18), Fraction(49, 19), Fraction(49, 20), Fraction(49, 22), Fraction(49, 27), Fraction(49, 29), Fraction(49, 30), Fraction(49, 31), Fraction(49, 34), Fraction(49, 36), Fraction(49, 41), Fraction(49, 43), Fraction(64, 7), Fraction(64, 9), Fraction(64, 15), Fraction(64, 17), Fraction(64, 19), Fraction(64, 23), Fraction(64, 25), Fraction(64, 27), Fraction(64, 37), Fraction(64, 39), F